In [ ]:
# default_exp tex

In [ ]:
# hide
from py2gift.util import render_latex

In [ ]:
# export

from typing import Union, Iterable, List, Optional

import numpy as np

from py2gift.util import to_formula_maybe

# TeX

> A collection of functions to ease the inclusion of TeX strings in Python source code

# Definitions of functions

## Gaussian pdf

In [ ]:
# export

@to_formula_maybe
def gaussian_pdf(x: str = 'x', mean: str = r'\mu', variance: str = r'\sigma^2') -> str:
    """
        Returns a string representing the probability density function for a Gaussian distribution.
    """

    return r'\frac{1}{\sqrt{2\pi ' + variance + r'}}e^{-\frac{(' + x + '-' + mean + r')^2}{2' + variance + r'}}'

With no arguments, it yields the usual formula,

In [ ]:
render_latex(gaussian_pdf(to_formula=True))

The arguments allow to use different symbols for the random variable, the mean and the variance

In [ ]:
render_latex(gaussian_pdf(x="n", mean="m", variance="v", to_formula=True))

## Q-function

In [ ]:
# export

@to_formula_maybe
def q_function_approximation(x: str = 'x') -> str:
    """
    Returns a string representing the Stirling approximation.
    """

    return f'Q({x}) \\approx \\frac{{1}}{{2}} e^{{-\\frac{{{x}^2}}{{2}}}}'

With no arguments, the argument is $x$

In [ ]:
render_latex(q_function_approximation(to_formula=True))

but a specific variable can be passed

In [ ]:
render_latex(q_function_approximation('t', to_formula=True))

# Number

In [ ]:
# export

@to_formula_maybe
def from_number(n: Union[int, float], prefix: str = '', precision: int = 3) -> str:
    """
    Returns a string for a given number.
    """

    if (type(n) == float) or (type(n) == np.float64):

        format_specifier = f'.{precision}f'

        return f'{prefix}{n:{format_specifier}}'

    else:

        return f'{n}'

Valid for both floating point numbers,

In [ ]:
render_latex(from_number(2.3, to_formula=True))

and integers

In [ ]:
render_latex(from_number(3, to_formula=True))

# Matrix/vector

In [ ]:
# export

@to_formula_maybe
def from_matrix(m: Union[list, np.ndarray], float_point_precision: int = 3) -> str:
    """
    Returns a string for a given array or matrix.
    """
    
    format_from_number = lambda x: f'.{float_point_precision}f' if (type(x) == np.float64) or (type(x) == float) else f'd'

    if isinstance(m[0], (list, np.ndarray)):

        return r'\begin{bmatrix}' + r' \\ '.join(
            [(r' & '.join([f'{e:{format_from_number(m[0][0])}}' for e in row])) for row in m]) + r'\end{bmatrix}'

    else:
        
        return r'\begin{bmatrix}' + r' & '.join([f'{e:{format_from_number(m[0])}}' for e in m]) + r'\end{bmatrix}'

This can be applied on lists

In [ ]:
render_latex(from_matrix([1, 2, 3], to_formula=True))

or matrices

In [ ]:
render_latex(from_matrix(np.array([[1.11, 3.14], [14.2, 5.1]]), to_formula=True))

In [ ]:
render_latex(from_matrix(np.array([14.2, 5.1]), to_formula=True))

## Integer matrix

In [ ]:
# # export

# @to_formula_maybe
# def from_integer_matrix(m: Iterable) -> str:
#     """
#     Returns a string for a given matrix of integers
#     """

#     return r'\begin{bmatrix}' + r' \\ '.join([(r' & '.join([f'{e:d}' for e in row])) for row in m]) + r'\end{bmatrix}'

In [ ]:
render_latex(from_matrix(np.array([[1, 3], [4, 5]]), to_formula=True))

# Enumerations

In [ ]:
# export

def enumerate_math(
    numbers_list: List[float], assigned_to: Optional[str] = None, nexus: str = 'and', precision: int = 3) -> str:
    """
    Returns a string for a enumeration of formulas.
    """

    format_specifier = f'.{precision}f'

    strings_list = [f'{e:{format_specifier}}' for e in numbers_list]

    if assigned_to:

        strings_list = [assigned_to.format(i_s+1) + ' = ' + s for i_s, s in enumerate(strings_list)]

    return '$' + '$, $'.join(strings_list[:-1]) + f'$ {nexus} ${strings_list[-1]}$'

* `numbers_list` is a list of `float`s
* `assigned_to` is a string with a [replacement field](https://docs.python.org/3/library/string.html#format-string-syntax)

In [ ]:
render_latex(enumerate_math([0.7, 0.9], assigned_to='w_t^{{({})}}'))

In [ ]:
# export

def expand(template: str, n: int, to_math: bool = False, nexus: str = 'and') -> str:
    """
    Expand a symbol according to a pattern.

    >>> util.expand('s_{}', 3, True)
    '$s_1$, $s_2$ and $s_3$'

    Parameters
    ----------
    template : str
        String with a *single* replacement field ({})
    n : int
        Requested number of terms
    to_math : bool
        If `True`, every output term is enclosed between $'s
    nexus : str
        String joining the second to last and last terms.

    Returns
    -------

    """

    res = [template.format(i) for i in range(1, 1 + n)]

    if to_math:

        res = [f'${e}$' for e in res]

    return ', '.join(res[:-1]) + f' {nexus} {res[-1]}'

assert expand('s_{}', 3, True) == '$s_1$, $s_2$ and $s_3$'

In [ ]:
render_latex(expand('s_{}', 3, True))